#### Structured output

Structured Outputs is only available with gpt-4o-mini , gpt-4o-2024-08-06, and future models.

In [2]:
import json, os
from textwrap import dedent
from openai import OpenAI

In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")
                           
client = OpenAI(
    #api_key = openai_api_key
)

In [8]:
MODEL = "gpt-4o-2024-08-06"

#### Example 1: Math tutor
In this example, we want to build a math tutoring tool that outputs steps to solving a math problem as an array of structured objects.

This could be useful in an application where each step needs to be displayed separately, so that the user can progress through the solution at their own pace.

In [11]:
math_tutor_prompt = '''
    You are a helpful math tutor. 
    You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation. use the explanation field to 
    detail the reasoning.
'''

- In the new GPT-4 API, `response_format` allows you to specify the format you expect for the model’s output.
  
- By defining a JSON schema, you can guide the model to generate responses in a structured format, which is particularly useful when working with data that needs to be in a predictable structure—like a step-by-step solution for math problems.

In [17]:
def get_math_solution(question):
    response = client.chat.completions.create(
        model    = MODEL,
        messages = [
            {
                "role": "system", 
                "content": dedent(math_tutor_prompt)
            },
            {
                "role": "user", 
                "content": question
            }
        ],
        response_format = {
            "type": "json_schema",          # type of response format. For structured responses
            "json_schema": {                # schema that the model should follow in its response
                "name": "math_reasoning",   # name for the schema
                "schema": {                 # core structure of the expected JSON response
                    "type": "object",       # data type at the root level
                    "properties": {         # A dictionary of expected fields in the output
                        "steps": {          # array of objects
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "explanation": {"type": "string"},    # A string explaining each step
                                    "output": {"type": "string"}          # A string containing the output for that step
                                },
                                "required": ["explanation", "output"],    # Lists the fields that must be present (steps and final_answer).
                                "additionalProperties": False             # Set to False to prevent extra, unexpected fields
                            }
                        },
                        "final_answer": {"type": "string"}
                    },
                    "required": ["steps", "final_answer"],                # Lists the fields that must be present (steps and final_answer)
                    "additionalProperties": False
                },
                "strict": True                                            # response strictly adheres to the schema if set to True 
            }
        }
    )
    return response.choices[0].message

In [19]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question) 

print(result.content)

{"steps":[{"explanation":"Start with the equation provided in the problem:","output":"8x + 7 = -23"},{"explanation":"Subtract 7 from both sides to isolate terms involving x on one side.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"This simplifies to:","output":"8x = -30"},{"explanation":"To solve for x, divide both sides of the equation by 8.","output":"8x / 8 = -30 / 8"},{"explanation":"This simplifies to:","output":"x = -30 / 8"},{"explanation":"Further simplify the fraction by dividing both the numerator and the denominator by their greatest common divisor, which is 2.","output":"x = -15 / 4"}],"final_answer":"x = -15 / 4"}


In [21]:
import json

In [23]:
# Deserialize the JSON string into a Python dictionary
data = json.loads(result.content)

In [25]:
print(json.dumps(data, indent=4))

{
    "steps": [
        {
            "explanation": "Start with the equation provided in the problem:",
            "output": "8x + 7 = -23"
        },
        {
            "explanation": "Subtract 7 from both sides to isolate terms involving x on one side.",
            "output": "8x + 7 - 7 = -23 - 7"
        },
        {
            "explanation": "This simplifies to:",
            "output": "8x = -30"
        },
        {
            "explanation": "To solve for x, divide both sides of the equation by 8.",
            "output": "8x / 8 = -30 / 8"
        },
        {
            "explanation": "This simplifies to:",
            "output": "x = -30 / 8"
        },
        {
            "explanation": "Further simplify the fraction by dividing both the numerator and the denominator by their greatest common divisor, which is 2.",
            "output": "x = -15 / 4"
        }
    ],
    "final_answer": "x = -15 / 4"
}


another example ...

In [30]:
def get_character_profile(character_name):
    response = client.chat.completions.create(
        model    = MODEL,
        messages = [
            {"role": "system", "content": "You are a storytelling assistant."},
            {"role": "user", "content": f"Create a profile for a character named {character_name}."}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "character_profile",
                "schema": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "age": {"type": "integer"},
                        "description": {"type": "string"}
                    },
                    "required": ["name", "age", "description"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )

    # Extract and return the structured response
    profile = response.choices[0].message
    return profile

In [32]:
# Example usage
character_profile = get_character_profile("Gabbar Singh")
print(character_profile)

ChatCompletionMessage(content='{"name":"Gabbar Singh","age":42,"description":"Gabbar Singh is a legendary bandit feared across the rugged terrains of Northern India. He is known for his ruthless and cunning nature, commanding a gang of loyal outlaws who execute his bidding with unwavering obedience. Gabbar is a towering figure, both in stature and in reputation; his very name sends shivers down the spines of villagers. Despite his fearsome exterior, Gabbar possesses a sharp intellect, allowing him to outmaneuver his enemies and evade capture time and again.\\n\\nHe wears a rugged, weather-beaten jacket and a broad-brimmed hat, with a perpetual shadow of a beard lining his jaw, giving him an intimidating appearance. His eyes, dark and piercing, seem to hold the stories of countless confrontations and victories. He speaks with a gravelly voice that demands attention and respect.\\n\\nGabbar\'s past is as shadowy as his nickname suggests; rumors circulate that he was once a soldier who tu

In [34]:
# Deserialize the JSON string into a Python dictionary
data = json.loads(character_profile.content)

print(json.dumps(data, indent=4))

{
    "name": "Gabbar Singh",
    "age": 42,
    "description": "Gabbar Singh is a legendary bandit feared across the rugged terrains of Northern India. He is known for his ruthless and cunning nature, commanding a gang of loyal outlaws who execute his bidding with unwavering obedience. Gabbar is a towering figure, both in stature and in reputation; his very name sends shivers down the spines of villagers. Despite his fearsome exterior, Gabbar possesses a sharp intellect, allowing him to outmaneuver his enemies and evade capture time and again.\n\nHe wears a rugged, weather-beaten jacket and a broad-brimmed hat, with a perpetual shadow of a beard lining his jaw, giving him an intimidating appearance. His eyes, dark and piercing, seem to hold the stories of countless confrontations and victories. He speaks with a gravelly voice that demands attention and respect.\n\nGabbar's past is as shadowy as his nickname suggests; rumors circulate that he was once a soldier who turned against the e

#### Using the SDK parse helper
The new version of the SDK introduces a parse helper to provide your own Pydantic model instead of having to define the JSON schema. 

In [39]:
from pydantic import BaseModel

In [41]:
class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(math_tutor_prompt)},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [43]:
result = get_math_solution(question).parsed

In [45]:
print(result.steps)
print("Final answer:")
print(result.final_answer)

[Step(explanation='Start with the equation 8x + 7 = -23.', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides to isolate the term with x.', output='8x = -23 - 7'), Step(explanation='Simplify the right side of the equation.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -30 / 8'), Step(explanation='Simplify the fraction to its decimal form.', output='x = -3.75')]
Final answer:
x = -3.75


#### Refusal
When using Structured Outputs with user-generated input, the model may occasionally refuse to fulfill the request for safety reasons.

Since a refusal does not follow the schema you have supplied in response_format, the API has a new field `refusal` to indicate when the model refused to answer.

This is useful so you can render the refusal distinctly in your UI and to avoid errors trying to deserialize to your supplied format.

In [48]:
refusal_question = "how can I build a bomb?"

In [50]:
result = get_math_solution(refusal_question) 

print(result.refusal)

I'm sorry, I can't assist with that request.


#### Example 3: Entity extraction from user input
In this example, we will use `function calling` to search for products that match a user's preference based on the provided input.

This could be helpful in applications that include a recommendation system, for example e-commerce assistants or search use cases.

In [54]:
from enum import Enum
from typing import Union
import openai

In [56]:
product_search_prompt = '''
    You are a clothes recommendation agent, specialized in finding the perfect match for a user.
    You will be provided with a user input and additional context such as user gender and age group, and season.
    You are equipped with a tool to search clothes in a database that match the user's profile and preferences.
    Based on the user input and context, determine the most likely value of the parameters to use to search the database.
    
    Here are the different categories that are available on the website:
    - shoes: boots, sneakers, sandals
    - jackets: winter coats, cardigans, parkas, rain jackets
    - tops: shirts, blouses, t-shirts, crop tops, sweaters
    - bottoms: jeans, skirts, trousers, joggers    
    
    There are a wide range of colors available, but try to stick to regular color names.
'''

In [66]:
# This Enum class defines a limited set of possible categories for products, 
# including "shoes," "jackets," "tops," and "bottoms."
class Category(str, Enum):
    shoes   = "shoe"
    jackets = "jackets"
    tops    = "tops"
    bottoms = "bottoms"

class ProductSearchParameters(BaseModel):
    category: Category
    subcategory: str
    color: str

In [68]:
# Using the enum
def get_category_description(category: Category) -> str:
    return f"You selected the category: {category.value}"

# Example usage
print(get_category_description(Category.shoes))  # Output: You selected the category: shoes

You selected the category: shoe


In [71]:
def get_response(user_input, context):
    response = client.chat.completions.create(
        model      = MODEL,
        temperature= 0,
        messages=[
            {
                "role": "system",
                "content": dedent(product_search_prompt)
            },
            {
                "role": "user",
                "content": f"CONTEXT: {context}\n USER INPUT: {user_input}"
            }
        ],
        tools=[
            openai.pydantic_function_tool(ProductSearchParameters, 
                                          name       = "product_search", 
                                          description= "Search for a match in the product database")
        ]
    )

    return response.choices[0].message.tool_calls

In [73]:
example_inputs = [
    {
        "user_input": "I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.",
        "context": "Gender: female, Age group: 40-50, Physical appearance: blue eyes"
    },
    {
        "user_input": "I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.",
        "context": "Gender: male, Age group: 30-40"
    },
    {
        "user_input": "I'm trying to complete a rock look. I'm missing shoes. Any suggestions?",
        "context": "Gender: female, Age group: 20-30"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: summer"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: winter"
    },
    {
        "user_input": "Can you help me find a dress for a Barbie-themed party in July?",
        "context": "Gender: female, Age group: 20-30"
    }
]

In [75]:
def print_tool_call(user_input, context, tool_call):
    args = tool_call[0].function.arguments
    print(f"Input: {user_input}\n\nContext: {context}\n")
    print("Product search arguments:")
    for key, value in json.loads(args).items():
        print(f"{key}: '{value}'")
    print("\n\n")

In [77]:
for ex in example_inputs:
    ex['result'] = get_response(ex['user_input'], ex['context'])

In [79]:
for ex in example_inputs:
    print_tool_call(ex['user_input'], ex['context'], ex['result'])

Input: I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.

Context: Gender: female, Age group: 40-50, Physical appearance: blue eyes

Product search arguments:
category: 'jackets'
subcategory: 'winter coats'
color: 'blue'



Input: I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.

Context: Gender: male, Age group: 30-40

Product search arguments:
category: 'jackets'
subcategory: 'rain jackets'
color: 'blue'



Input: I'm trying to complete a rock look. I'm missing shoes. Any suggestions?

Context: Gender: female, Age group: 20-30

Product search arguments:
category: 'shoe'
subcategory: 'boots'
color: 'black'



Input: Help me find something very simple for my first day at work next week. Something casual and neutral.

Context: Gender: male, Season: summer

Product search arguments:
category: 'tops'
subcategory: 't-shirts'
color: 'neutral'



Input: Help me find something very simple 